<a href="https://colab.research.google.com/github/FAhtisham/Latext-based-EnhancerGAN/blob/main/EDGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [83]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
from torch.utils.data import Dataset, DataLoader


import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import collections

from tqdm import tqdm 

# Load the Dataset

In [78]:
def read_fasta( fp):
        name, seq = None, []
        for line in fp:
            line = line.rstrip()
            if line.startswith(">"):
                if name: yield (name, ''.join(seq))
                name, seq = line, []
            else:
                seq.append(line)
        if name: yield (name, ''.join(seq))



def load_data():
        sequences = []
        # Reading FASTA file
        with open("permissive_enhancers","r") as fp:
            for name, seq in read_fasta(fp):
                sequences.append(seq)
            print("Sequences Read Succesfully !!!!")
        print("Total Raw Sequences: ",len(sequences))
        return sequences

def add_padding(seq, p_len):
  seq = seq + ("P" * p_len)
  return seq
  

def convert_seqs_to_words(sequences):
  f_sequences = []
  for i in range(len(sequences)):
    temp = ""
    str_ = sequences[i]
    j=0

    if len(str_)%3!=0:
      n = len(str_)
      while n % 3 != 0:
        n+=1
      str_= add_padding(str_,n-len(str_))

    for k in range(0,len(str_)):
      j+=1
      if  j%3==0:
        temp = temp + str_[k-2:j] + ' ' 
        #j+=3
    f_sequences.append(temp) 
  f_sequences= [j.split() for j in f_sequences]
  return f_sequences

In [79]:
f_sequences = convert_seqs_to_words(load_data())
print("Total Processed Sequences: ",len(f_sequences))

Sequences Read Succesfully !!!!
Total Raw Sequences:  43011
Total Processed Sequences:  43011


In [80]:
len(f_sequences)

43011

# Dataset Preprocessing

In [92]:
class Nucleotides:
  def __init__(self, seqs):
    self.nuc_pairs= self.make_pairs(seqs)
    self.encoding= {w:i for i,w in enumerate(self.nuc_pairs,1)}
    self.decoding= {i:w for i,w in enumerate(self.nuc_pairs,1)}

  #def 
  def make_pairs(self, seqs, clip=1):
    nuc_pairs= collections.Counter()

    for seq in tqdm(seqs):
      nuc_pairs.update(seq)

    # check why this statement is so important (84, remains same without it)
    for nucs in list(nuc_pairs.keys()):
      if nuc_pairs[nucs] < clip:
        nuc_pairs.pop(nucs)

    return list(sorted(nuc_pairs.keys()))
  


class Sequences(Dataset):
  def __init__(self):
    print("done")


In [95]:
nuc_pairs = Nucleotides(f_sequences).nuc_pairs
print(words, "\n", len(words))
encoding = Nucleotides(f_sequences).encoding
print(encoding)

 21%|██        | 9000/43011 [00:00<00:00, 89998.58it/s]

['AAA', 'AAC', 'AAG', 'AAP', 'AAT', 'ACA', 'ACC', 'ACG', 'ACP', 'ACT', 'AGA', 'AGC', 'AGG', 'AGP', 'AGT', 'APP', 'ATA', 'ATC', 'ATG', 'ATP', 'ATT', 'CAA', 'CAC', 'CAG', 'CAP', 'CAT', 'CCA', 'CCC', 'CCG', 'CCP', 'CCT', 'CGA', 'CGC', 'CGG', 'CGP', 'CGT', 'CPP', 'CTA', 'CTC', 'CTG', 'CTP', 'CTT', 'GAA', 'GAC', 'GAG', 'GAP', 'GAT', 'GCA', 'GCC', 'GCG', 'GCP', 'GCT', 'GGA', 'GGC', 'GGG', 'GGP', 'GGT', 'GPP', 'GTA', 'GTC', 'GTG', 'GTP', 'GTT', 'TAA', 'TAC', 'TAG', 'TAP', 'TAT', 'TCA', 'TCC', 'TCG', 'TCP', 'TCT', 'TGA', 'TGC', 'TGG', 'TGP', 'TGT', 'TPP', 'TTA', 'TTC', 'TTG', 'TTP', 'TTT'] 
 84


100%|██████████| 43011/43011 [00:00<00:00, 92089.08it/s]

{'AAA': 1, 'AAC': 2, 'AAG': 3, 'AAP': 4, 'AAT': 5, 'ACA': 6, 'ACC': 7, 'ACG': 8, 'ACP': 9, 'ACT': 10, 'AGA': 11, 'AGC': 12, 'AGG': 13, 'AGP': 14, 'AGT': 15, 'APP': 16, 'ATA': 17, 'ATC': 18, 'ATG': 19, 'ATP': 20, 'ATT': 21, 'CAA': 22, 'CAC': 23, 'CAG': 24, 'CAP': 25, 'CAT': 26, 'CCA': 27, 'CCC': 28, 'CCG': 29, 'CCP': 30, 'CCT': 31, 'CGA': 32, 'CGC': 33, 'CGG': 34, 'CGP': 35, 'CGT': 36, 'CPP': 37, 'CTA': 38, 'CTC': 39, 'CTG': 40, 'CTP': 41, 'CTT': 42, 'GAA': 43, 'GAC': 44, 'GAG': 45, 'GAP': 46, 'GAT': 47, 'GCA': 48, 'GCC': 49, 'GCG': 50, 'GCP': 51, 'GCT': 52, 'GGA': 53, 'GGC': 54, 'GGG': 55, 'GGP': 56, 'GGT': 57, 'GPP': 58, 'GTA': 59, 'GTC': 60, 'GTG': 61, 'GTP': 62, 'GTT': 63, 'TAA': 64, 'TAC': 65, 'TAG': 66, 'TAP': 67, 'TAT': 68, 'TCA': 69, 'TCC': 70, 'TCG': 71, 'TCP': 72, 'TCT': 73, 'TGA': 74, 'TGC': 75, 'TGG': 76, 'TGP': 77, 'TGT': 78, 'TPP': 79, 'TTA': 80, 'TTC': 81, 'TTG': 82, 'TTP': 83, 'TTT': 84}


# Autoencoder Model

In [104]:
class Autoencoder(nn.Module):
  def __init__(self, nuc_pair_size, embedding_dims, e_hidden_dims, bottleneck_dims, dec_hidden_dims, dropout_size = 0.2):
    super().__init__()
    
    # define the vars over here (layers, objects)
    self.embedding= nn.Embedding(num_embeddings= nuc_pair_size, embedding_dim= embedding_dims)
    self.rnn1= nn.LSTM(input_size= embedding_dims, hidden_size= e_hidden_dims)
    self.fc1= nn.Linear(input= e_hidden_dims, out_features= bottleneck_dims)
    self.a1= nn.ReLU(True)
    self.dropout= nn.Dropout(dropout_size)

    self.fc2= nn.Linear(input= bottleneck_dims, output_features= d_hidden_dims)
    self.rnn2= nn.LSTM(input_size= dec_hidden_dims, hidden_size= d_hidden_dims)
    self.fc3= nn.Linear(input= d_hidden_dims, output_features= nuc_pair_size)



  def encoder(self, x):
    x= self.embedding(x)
    _,(hidden_states, _)= self.rnn1(x)
    lv= self.fc1(x) # latent vector
    lv= self.Dropout(lv)
    return lv


  def decoder(self, lv):
    lv= self.fc2(lv)
    output, _= self.rnn2(lv)
    logits= self.fc3(output)
    return logits
  
  def forward(self,x):
    lv= self.encoder(x)
    logits= self.decoder(lv)
    return (lv.squeeze(), logits)

In [105]:
def load(batch_size, seq_len):
    return (DataLoader(f_sequences, batch_size, shuffle=True), nuc_pairs)

# Training Loop


In [106]:
import argparse


def train(epoch):
  model.train()
  train_loss=0
  for i,x in enumerate(train_loader):
    optimizer.zero_grad()
    
    x = x.to(device)
    _, logits= model(x)
    loss= criterion(logits, x)
    train_loss+=loss.item()
    loss.backward()
    optimizer.step()

    if args.interval > 0 and i % interval ==0:
      print("epoch: {} | Batch:{}/{} ({:0.f}%)| Loss:{:.6f}".format(
          epoch, args.batch_size*i, len(train_loader.dataset),
          100.*(args.batch_size*i)/len(train_loader.dataset),
          loss.item
        
      ))

  train_loss /= len(train_loader)
  print('(Train) Epoch: {}|loss{:.4f}'.format(epoch, train_loss))
  return train_loss







In [107]:
'''
parser = argparse.ArgumentParser()
parser.add_argument("--seed", type=int, default=0)
parser.add_argument("--epochs", type=int, default=10)
parser.add_argument("--batch-size", type=int, default=32)
parser.add_argument("--lr", type=float, default=5e-4)
parser.add_argument("--dropout", type=int, default=0.2)
parser.add_argument("--embedding-dim", type=int, default=200)
parser.add_argument("e-hidden-dim", type=int, default=100)
args = parser.parse_args()

print(args)
'''

seed= 0
spochs= 5
batch_size= 32
lr= 5e-04
dropout= 0.2
embedding_dim=200
e_hidden_dims= 1
d_hidden_dims=600
seq_length= 951

torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

train_loader, nuc_pairs= load(batch_size, seq_length)
model = Autoencoder( nuc_pair_size, embedding_dims, e_hidden_dims, bottleneck_dims, dec_hidden_dims, dropout_size = 0.2)

TypeError: ignored

# Practice

In [39]:
arr = torch.tensor([[1,2],[3,4],[5,6]])
arr.size()
arr = arr.permute(1,0)
arr.size()
arr

tensor([[1, 3, 5],
        [2, 4, 6]])

In [38]:
arr2 = torch.randn(3,5,2)
print(arr2.size())
print(arr2)
arr2= arr2.permute(2,0,1)
print(arr2.size())
print(arr2)

torch.Size([3, 5, 2])
tensor([[[ 0.2006,  1.8473],
         [ 0.1055, -1.1586],
         [ 0.1208,  0.3094],
         [-0.0632, -0.5114],
         [ 0.2881,  1.3463]],

        [[-0.3929,  1.1025],
         [ 1.2375,  0.4721],
         [-0.2026,  0.5434],
         [-0.2871, -0.9966],
         [-0.8792, -0.4815]],

        [[-1.4430, -0.9805],
         [ 0.2667,  0.6952],
         [-0.1730, -1.1442],
         [ 0.8840,  0.5541],
         [-0.0819, -0.5837]]])
torch.Size([2, 3, 5])
tensor([[[ 0.2006,  0.1055,  0.1208, -0.0632,  0.2881],
         [-0.3929,  1.2375, -0.2026, -0.2871, -0.8792],
         [-1.4430,  0.2667, -0.1730,  0.8840, -0.0819]],

        [[ 1.8473, -1.1586,  0.3094, -0.5114,  1.3463],
         [ 1.1025,  0.4721,  0.5434, -0.9966, -0.4815],
         [-0.9805,  0.6952, -1.1442,  0.5541, -0.5837]]])
